In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sacred
from tqdm import tqdm

import librosa
import os
import random

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [146]:
ex = sacred.Experiment('MLP', interactive=True)
ex.observers.append(sacred.observers.FileStorageObserver.create('ex'))

In [147]:
@ex.config
def cfg():
    SR = 32000
    N_MFCC = 13
    ROOT_DIR = './huge'
    N_CLASSES = 2
    BATCH_SIZE = 128
    EPOCHS = 100
    LEARNING_RATE = 3e-4
    SEED = 42
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    TRAIN_MODE = True

In [148]:
@ex.capture
def set_seed(SEED, DEVICE):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if DEVICE == 'cuda':
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [149]:
@ex.capture
def get_mfcc_feature(df, SR, N_MFCC, ROOT_DIR, N_CLASSES, TRAIN_MODE):
    features = []
    labels = []
    
    for _, row in df.iterrows():
        y, sr = librosa.load(os.path.join(ROOT_DIR, row['path']), sr=SR)
        
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)
        
        if TRAIN_MODE:
            label = row['label']
            label_vector = np.zeros(N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)
            
    if TRAIN_MODE :
        return features, labels
    
    return features

In [150]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label
        
    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, idx):
        if self.label is not None:
            return self.mfcc[idx], self.label[idx]
        return self.mfcc[idx]

In [151]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x


In [152]:
@ex.capture
def train_model(model, optimizer, train_loader, val_loader, DEVICE, EPOCHS):
    model.to(DEVICE)
    criterion = nn.BCELoss().to(DEVICE)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = []
        for features, labels in iter(train_loader):
            features = features.float().to(DEVICE)
            labels = labels.float().to(DEVICE)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validate_model(model, criterion, val_loader, DEVICE)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [153]:
@ex.capture
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [154]:
@ex.capture
def validate_model(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in iter(val_loader):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

In [155]:
@ex.capture
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    
    return predictions

In [156]:
@ex.main
def my_main(_config, _run):
    set_seed(_config['SEED'])

    df = pd.read_csv(os.path.join(_config['ROOT_DIR'], 'train.csv'))
    train_df, val_df, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=_config['SEED'])
    
    print('--- get mfcc feature ---')
    train_mfcc, train_labels = get_mfcc_feature(train_df)
    val_mfcc, val_labels = get_mfcc_feature(val_df)
    
    print('--- create dataset ---')
    train_dataset = CustomDataset(train_mfcc, train_labels)
    val_dataset = CustomDataset(val_mfcc, val_labels)

    print('--- create dataloader ---')
    train_loader = DataLoader(train_dataset, batch_size=_config['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=_config['BATCH_SIZE'], shuffle=False)
    
    print('--- create model ---')
    model = MLP(input_dim=_config['N_MFCC'])
    optimizer = torch.optim.Adam(params=model.parameters(), lr=_config['LEARNING_RATE'])
    
    print('--- train model ---')
    best_model = train_model(model, optimizer, train_loader, val_loader)
    
    print('--- inference ---')
    test_df = pd.read_csv(os.path.join(_config['ROOT_DIR'], 'test.csv'))
    test_mfcc = get_mfcc_feature(test_df, train_mode=False)
    test_dataset = CustomDataset(test_mfcc, None)
    test_loader = DataLoader(test_dataset, batch_size=_config['BATCH_SIZE'], shuffle=False)
    
    preds = inference(best_model, test_loader, _config['DEVICE'])
    
    print('--- save submission ---')
    submit = pd.read_csv(os.path.join(_config['ROOT_DIR'], 'sample_submission.csv'))
    submit.iloc[:, 1:] = preds
    submit.to_csv('./data/output/baseline_submit.csv', index=False)

In [157]:
ex.run()

INFO - MLP - Running command 'my_main'
INFO - MLP - Started run with ID "2"


--- get mfcc feature ---


WARNING - MLP - Aborted after 0:00:31!


KeyboardInterrupt: 